Let's enhance the existing v1 LangGraph-based customer support agent with **LLM integration**, **rescheduling logic**, **conditional edges**, and a more sophisticated workflow. 

## Step 1: Integrate LLM for Intent/Entity Extraction

Replace the mock input parser with an LLM-powered system using OpenAI.

### 1.1 Install Dependencies

In [ ]:
!pip install openai python-dotenv

### 1.2 Add OpenAI Parser Node

In [ ]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def llm_parse_input(state: AgentState) -> AgentState:
    """Use LLM to extract intent/entities."""
    user_input = state["user_input"]
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "system",
            "content": """Extract intent and entities from travel queries:
            - Intent: cancel_booking, reschedule_booking, general_inquiry
            - Entities: booking_id, new_date, destination
            Return JSON only."""
        }, {
            "role": "user",
            "content": user_input
        }]
    )
    
    parsed = json.loads(response.choices[0].message.content)
    state.update(parsed)
    return state

## Step 2: Add Rescheduling Logic

### 2.1 Extend Mock API

Add a new endpoint:

In [ ]:
@app.post("/bookings/{booking_id}/reschedule")
def reschedule_booking(booking_id: str, new_date: str, api_key: str = Header(...)):
    if booking_id not in mock_bookings:
        raise HTTPException(status_code=404, detail="Booking not found")
    mock_bookings[booking_id]["date"] = new_date
    return {"message": f"Rescheduled to {new_date}"}

### 2.2 Add Rescheduling Nodes

In [ ]:
def check_availability(state: AgentState) -> AgentState:
    """Mock flight/hotel availability check."""
    state["is_available"] = True  # Replace with actual API call
    return state

def process_rescheduling(state: AgentState) -> AgentState:
    """Call reschedule API."""
    if state.get("confirmation") and state["is_available"]:
        response = requests.post(
            f"{ngrok_tunnel.public_url}/bookings/{state['booking_id']}/reschedule",
            headers={"api_key": state["api_key"]},
            json={"new_date": state["new_date"]}
        )
        if response.status_code != 200:
            state["error"] = "Rescheduling failed"
    return state

## Step 3: Enhance Graph with Conditional Edges

### 3.1 Intent-Based Routing

In [ ]:
def route_intent(state: AgentState) -> str:
    """Conditional edge: Route based on intent."""
    if state.get("error"):
        return "error_handler"
    intent = state.get("intent")
    if intent == "cancel_booking":
        return "confirm_cancellation"
    elif intent == "reschedule_booking":
        return "check_availability"
    else:
        return "general_inquiry_handler"

builder.add_conditional_edges(
    "authenticate",
    route_intent
)